<a href="https://colab.research.google.com/github/c90x/loan_prediction/blob/main/loan_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loan Dataset - Lab

## Dependencies

In [ ]:
import re
from pathlib import Path

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Configure theme

In [ ]:
plt.style.use("dark_background")

**Download** dataset from Kaggle

In [ ]:
import kagglehub

# Download latest version
path = Path(kagglehub.dataset_download("ranadeep/credit-risk-dataset"))

path = path / "loan" / "loan.csv"

assert path.exists()
assert path.is_file()

print("Downloaded dataset to path:", path)

Load dataset

In [ ]:
df = pd.read_csv(path)
df = df.sample(n=50000, random_state=99)

## Preprocessing

In [ ]:
print(df.columns)

Select columns

In [ ]:
df_select = [
    "loan_status",  # e
    #
    # 'addr_state',
    "annual_inc",
    "annual_inc_joint",
    #'application_type',
    "collection_recovery_fee",
    "collections_12_mths_ex_med",
    "dti",
    "dti_joint",
    "emp_length",  # e
    #'fico_range_high',
    #'fico_range_low',
    #'grade',
    "home_ownership",  # e
    #'is_inc_v',
    #'last_fico_range_high',
    #'last_fico_range_low',
    "loan_amnt",
    "open_acc",
    "pub_rec",
    "purpose",
    "recoveries",  #!!!
    "sub_grade",  # e
    "verification_status",  # e
    #'verified_status_joint',
    "open_acc_6m",
    "open_il_6m",
    "open_il_12m",
    "open_il_24m",
    "mths_since_rcnt_il",
    "total_bal_il",
    "il_util",
    "open_rv_12m",
    "open_rv_24m",
    "max_bal_bc",
    "all_util",
    "total_rev_hi_lim",
    #'inq_fi',
    #'total_cu_tl',
    #'inq_last_12m',
    "acc_now_delinq",
    "tot_coll_amt",
    "tot_cur_bal",
]

df = df[df_select]

In [ ]:
print(df.columns)

List **numerical data**

In [ ]:
df.select_dtypes(include=[np.number]).columns.tolist()

List **non numerical data**

In [ ]:
df.select_dtypes(include=["object"]).columns.tolist()

Encode `loan_status`

In [ ]:
df["loan_status"].value_counts()

In [ ]:
# loan_target:
# 0: bad
# 1: good

df_loan_status_good = [
    "current",
    "fully paid",
    "issued",
]


def df_loan_status_map(x):
    if "fully paid" in x.lower():
        return 1
    return 1 if x.lower() in df_loan_status_good else 0


df["loan_target"] = df["loan_status"].map(df_loan_status_map).fillna(0)

In [ ]:
df.drop("loan_status", axis=1, inplace=True)

Encode `emp_length`

In [ ]:
df["emp_length"].unique()

In [ ]:
def df_emp_length_map(s):
    if s.lower() == "< 1 year":
        return 0

    m = re.search(r"\d+", s)
    return int(m.group()) if m else 0


df["emp_length"] = df["emp_length"].map(df_emp_length_map, na_action="ignore").fillna(0)

One hot encode `home_ownership`

In [ ]:
df = pd.get_dummies(df, columns=["home_ownership"], drop_first=True)

Target encode `purpose`

In [ ]:
# Calculate the mean of the target variable for each category in the 'purpose' column
df_purpose_target_mean = df.groupby("purpose")["loan_target"].mean()

# Map these means back to the original dataframe
df["purpose"] = df["purpose"].map(df_purpose_target_mean)

Encode `subgrade`

In [ ]:
df["sub_grade"].unique()

In [ ]:
def df_sub_grade_map(s):
    return (ord(s[0]) - ord("A") + 1) + (int(s[1]) - 1) / 5


df["sub_grade"] = df["sub_grade"].map(df_sub_grade_map)

One hot encode `verification_status`

In [ ]:
df = pd.get_dummies(df, columns=["verification_status"], drop_first=True)

Mark missing

In [ ]:
for c in [
    "open_acc_6m",
    "open_il_6m",
    "open_il_12m",
    "open_il_24m",
    "mths_since_rcnt_il",
    "total_bal_il",
    "il_util",
    "open_rv_12m",
    "open_rv_24m",
    "max_bal_bc",
    "all_util",
    "total_rev_hi_lim",
]:
    df[f"{c}_m"] = df[c].apply(lambda x: 0 if pd.isna(x) else 1)

Fill missing

In [ ]:
def df_fillna_with_median(c):
    df.fillna({c: df[c].median()}, inplace=True)


def df_fillna_with_median_of_loan(c):
    median_good = df[df["loan_target"] == 1][c].median()
    median_bad = df[df["loan_target"] == 0][c].median()

    for index, row in df[df[c].isnull()].iterrows():
        if row["loan_target"] == 1:
            df.at[index, c] = median_good
        else:
            df.at[index, c] = median_bad

In [ ]:
df_fillna_with_median_of_loan("annual_inc")
df.fillna({"annual_inc_joint": 0}, inplace=True)
assert df["collection_recovery_fee"].isnull().sum() == 0
df.fillna({"collections_12_mths_ex_med": 0}, inplace=True)
assert df["dti"].isnull().sum() == 0
df.fillna({"dti_joint": 0}, inplace=True)
assert df["loan_amnt"].isnull().sum() == 0
df_fillna_with_median_of_loan("open_acc")
df.fillna({"pub_rec": 0}, inplace=True)
assert df["recoveries"].isnull().sum() == 0
df.fillna({"open_acc_6m": -1}, inplace=True)  # m
df.fillna({"open_il_6m": -1}, inplace=True)  # m
df.fillna({"open_il_12m": -1}, inplace=True)  # m
df.fillna({"open_il_24m": -1}, inplace=True)  # m
df.fillna({"mths_since_rcnt_il": -1}, inplace=True)  # m
df.fillna({"total_bal_il": -1}, inplace=True)  # m
df.fillna({"il_util": -1}, inplace=True)  # m
df.fillna({"open_rv_12m": -1}, inplace=True)  # m
df.fillna({"open_rv_24m": -1}, inplace=True)  # m
df.fillna({"max_bal_bc": -1}, inplace=True)  # m
df.fillna({"all_util": -1}, inplace=True)  # m
df.fillna({"total_rev_hi_lim": 0}, inplace=True)
df_fillna_with_median("acc_now_delinq")
df_fillna_with_median_of_loan("tot_coll_amt")
df_fillna_with_median_of_loan("tot_cur_bal")

In [ ]:
# for c in [
#     'revol_util', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'annual_inc',
#     'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'acc_now_delinq'
# ]:
#     df_fillna_with_median_of_loan(c)

Mark missing and delete

In [ ]:
def df_mark(columns):
    for c in columns:
        df[f"{c}_m"] = df[c].apply(lambda x: 0 if pd.isna(x) else 1)


df_mark([])

Print columns with incomplete values

In [ ]:
print(df.columns[df.isnull().any()].tolist())

Ensure all cells are valid

In [ ]:
assert df.isnull().sum().sum() == 0, "DataFrame contains NaN values"

## Analysis

Amount of good vs bad loans

In [ ]:
loan_target_counts = df["loan_target"].value_counts(normalize=True) * 100
loan_target_counts.plot(
    kind="pie",
    autopct="%1.1f%%",
    labels=["Good Loans", "Bad Loans"],
    colors=["#66b3ff", "#ff9999"],
    startangle=90,
)
plt.title("Percentage of Loan Targets")
plt.ylabel("")
plt.show()

`reco`

## Lab Model

Split Data

In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target
X = df.drop(["loan_target"], axis=1)
y = df["loan_target"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=99
)

Create and train decision tree

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the random forest classifier
clf = RandomForestClassifier(random_state=99, n_estimators=300, max_depth=15)
clf.fit(X_train, y_train)

# Predict on the test data
clf_y_pred = clf.predict(X_test)

Create dummy classifier

In [ ]:
from sklearn.dummy import DummyClassifier

# Create and train the dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=99)
dummy_clf.fit(X_train, y_train)

# Predict on the test data
dummy_y_pred = dummy_clf.predict(X_test)

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Generate the classification report
report = classification_report(
    y_test, clf_y_pred, target_names=["Bad Loans", "Good Loans"]
)
print(report)

In [ ]:
# Evaluate the dummy classifier
dummy_report = classification_report(
    y_test, dummy_y_pred, target_names=["Bad Loans", "Good Loans"], zero_division=0
)
print(dummy_report)

List most important features

In [ ]:
# Get feature importances
clf_importances = clf.feature_importances_
clf_feature_names = X.columns

# Create a DataFrame for visualization
clf_feature_importances = pd.DataFrame(
    {"Feature": clf_feature_names, "Importance": clf_importances}
)
clf_feature_importances = clf_feature_importances.sort_values(
    by="Importance", ascending=False
).head(25)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(
    clf_feature_importances["Feature"],
    clf_feature_importances["Importance"],
    color="skyblue",
)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Top 25 Feature Importances")
plt.gca().invert_yaxis()
plt.show()